In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-darkgrid')
import warnings
warnings.filterwarnings('ignore')

import time
import argparse

import pickle
import os
import gc
import pandas
import lightgbm

/Users/hernando/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
train = pd.read_csv('/Users/hernando/Desktop/datamining1/assign2/2nd-assignment-dmt-2020/training_set_VU_DM.csv',sep=',')
test = pd.read_csv('/Users/hernando/Desktop/datamining1/assign2/2nd-assignment-dmt-2020/test_set_VU_DM.csv',sep=',')
# all_data = pd.concat((train,test)).reset_index(drop=True)

# Drop missing Value

In [4]:
def drop_columns_with_missing_data(
    df,
    threshold=0.9,
    ignore_values=[
        "visitor_hist_adr_usd",
        "visitor_hist_starrating",
        "srch_query_affinity_score",
    ],
):
    
    df['avg_comp_rate'] = df[['comp1_rate', 'comp2_rate', 'comp3_rate', 'comp4_rate', 'comp5_rate', 'comp6_rate', 'comp7_rate', 'comp8_rate']].mean(axis=1)
    df = df.drop(['comp1_rate', "comp1_inv", "comp1_rate_percent_diff", 'comp2_rate', "comp2_inv", "comp2_rate_percent_diff", 'comp3_rate', "comp3_inv", "comp3_rate_percent_diff", 'comp4_rate', "comp4_inv", "comp4_rate_percent_diff", 'comp5_rate', "comp5_inv", "comp5_rate_percent_diff", 'comp6_rate', "comp6_inv", "comp6_rate_percent_diff", 'comp7_rate', "comp7_inv", "comp7_rate_percent_diff", 'comp8_rate', "comp8_inv", "comp8_rate_percent_diff"], axis = 1)
    df = df.fillna(value = {"avg_comp_rate": 0}) 
    columns_to_drop = []

    for i in range(df.shape[1]):
        length_df = len(df)
        column_names = df.columns.tolist()
        number_nans = sum(df.iloc[:, i].isnull())
        if number_nans / length_df > threshold:
            if column_names[i] not in ignore_values:
                columns_to_drop.append(column_names[i])

    print(
        "Dropping columns {} because they miss more than {} of data.".format(
            columns_to_drop, threshold
        )
    )
    
    df["orig_destination_distance"].fillna(df["orig_destination_distance"].describe()[6], inplace=True)
    df['visitor_hist_starrating'].fillna(df['visitor_hist_starrating'].median(axis = 0, skipna=True) ,inplace=True)
    df['visitor_hist_adr_usd'].fillna(df['visitor_hist_adr_usd'].median(axis = 0, skipna=True),inplace=True)
    df["srch_query_affinity_score"].fillna(df["srch_query_affinity_score"].min(), inplace=True)
    df['roomcount_bookwindow'] = df['srch_room_count']*max(df['srch_booking_window']) + df['srch_booking_window']
    df['adultcount_childrencount'] = df['srch_adults_count']*max(df['srch_children_count']) + df['srch_children_count']

    df_reduced = df.drop(labels=columns_to_drop, axis=1)
    
    
    print("Dropped columns {}".format(columns_to_drop))
    return df_reduced

In [5]:
train_change = drop_columns_with_missing_data(train)
test_change = drop_columns_with_missing_data(test)

Dropping columns ['gross_bookings_usd'] because they miss more than 0.9 of data.
Dropped columns ['gross_bookings_usd']
Dropping columns [] because they miss more than 0.9 of data.
Dropped columns []


In [6]:
train_change.shape

(4958347, 32)

In [7]:
test_change.shape

(4959183, 29)

In [8]:
all_data_na = (train_change.isnull().sum() / len(train_change)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data

,Missing Ratio
prop_location_score2,21.990151
prop_review_score,0.148517


In [9]:
all_data_na = (test_change.isnull().sum() / len(test_change)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data

,Missing Ratio
prop_location_score2,21.939743
prop_review_score,0.146516


# Fill NA

In [10]:
def normalize_features(input_df, group_key, target_column, take_log10=False):
    # for numerical stability
    epsilon = 1e-8
    if take_log10:
        input_df[target_column] = np.log10(input_df[target_column] + epsilon)
    methods = ["mean", "std"]

    df = input_df.groupby(group_key).agg({target_column: methods})

    df.columns = df.columns.droplevel()
    col = {}
    for method in methods:
        col[method] = target_column + "_" + method
    
    df.rename(columns=col, inplace=True)
    df_merge = input_df.merge(df.reset_index(), on=group_key)
    df_merge[target_column + "_norm_by_" + group_key] = (
        df_merge[target_column] - df_merge[target_column + "_mean"]
    ) / df_merge[target_column + "_std"]
    df_merge = df_merge.drop(labels=[col["mean"], col["std"]], axis=1)
    gc.collect()
    
    return df_merge



def aggregated_features_single_column(
    in_data,
    key_for_grouped_by="prop_id",
    target_column="price_usd",
    agg_methods=["mean", "median", "min", "max"],
    transform_methods={"mean": ["substract"]},
):
    df = in_data.groupby(key_for_grouped_by).agg({target_column: agg_methods})

    if isinstance(key_for_grouped_by, list):
        str_key_for_grouped_by = "|".join(key_for_grouped_by)
    else:
        str_key_for_grouped_by = key_for_grouped_by

    df.columns = df.columns.droplevel()
    col = {}
    for method in agg_methods:
        col[method] = (
            method.upper() + "(" + str_key_for_grouped_by + "_" + target_column + ")"
        )

    df.rename(columns=col, inplace=True)

    in_data = in_data.merge(df.reset_index(), on=key_for_grouped_by)
    for method_name in transform_methods:
        for applying_function in transform_methods[method_name]:
            function_data = in_data[
                method_name.upper()
                + "("
                + str_key_for_grouped_by
                + "_"
                + target_column
                + ")"
            ]
            column_data = in_data[target_column]
            if applying_function == "substract":
                result = column_data - function_data
            elif applying_function == "divide":
                result = column_data / function_data
            else:
                continue
            in_data[
                applying_function.upper()
                + "("
                + target_column
                + "_"
                + method_name.upper()
                + ")"
            ] = result
    gc.collect()

    return in_data



def data_processing(orig_data, kind="train", use_ndcg_choices=False):
    
    print("Preprocessing training data....")
    gc.collect()
    data_training = orig_data
    
    target_column = "target"
    
    ##select the target column
    if kind == "train":
        conditions = [data_training["click_bool"] == 1,data_training["booking_bool"] == 1]
        choices = [1, 2]
        data_training[target_column] = np.select(conditions, choices, default=0)
    
    ## Add date column
    dates = pandas.to_datetime(data_training["date_time"])
    data_training["month"] = dates.dt.month
    data_training["dayofweek"] = dates.dt.dayofweek
    data_training["hour"] = dates.dt.hour
#     data_training.drop(labels=["date_time"], axis=1, inplace=True)
    
    ###normal
    # do not normalize 2 times with take_log10
    data_training = normalize_features(data_training, group_key="srch_id",target_column="price_usd",take_log10=True,)
    
    data_training = normalize_features(
        data_training, group_key="prop_id", target_column="price_usd")
    
    data_training = normalize_features(
        data_training, group_key="srch_id", target_column="prop_starrating")
    
    #### aggre new feature
    data_training = aggregated_features_single_column(
        data_training, "prop_id", "price_usd", ["mean"])
    
    
    data_training = aggregated_features_single_column(
        data_training,
        key_for_grouped_by="srch_id",
        target_column="prop_starrating",
        agg_methods=["mean"],
        transform_methods={"mean": ["substract"]},)
    
    
    data_training = aggregated_features_single_column(
        data_training,
        key_for_grouped_by="srch_id",
        target_column="prop_location_score2",
        agg_methods=["mean"],
        transform_methods={"mean": ["substract"]},)
    
    data_training = aggregated_features_single_column(
        data_training,
        key_for_grouped_by="srch_id",
        target_column="prop_location_score1",
        agg_methods=["mean"],
        transform_methods={"mean": ["substract"]},)
    
    data_training = aggregated_features_single_column(
        data_training,
        key_for_grouped_by="srch_destination_id",
        target_column="price_usd",
        agg_methods=["mean"],
        transform_methods={"mean": ["substract"]},)
    
    data_training = aggregated_features_single_column(
        data_training,
        key_for_grouped_by="srch_id",
        target_column="prop_review_score",
        agg_methods=["mean"],
        transform_methods={"mean": ["substract"]},)
    
    data_training = aggregated_features_single_column(
        data_training,
        key_for_grouped_by="srch_id",
        target_column="promotion_flag",
        agg_methods=["mean"],
        transform_methods={"mean": ["substract"]},)
    # NOTE: has to be done after aggregated_features_single_column
    data_training = data_training.sort_values("srch_id")

    data_training = normalize_features(
        data_training, group_key="srch_id", target_column="prop_starrating"
    )
    data_training = normalize_features(
        data_training, group_key="srch_id", target_column="prop_location_score2"
    )
    data_training = normalize_features(
        data_training, group_key="srch_id", target_column="prop_location_score1"
    )
    data_training = normalize_features(
        data_training, group_key="srch_id", target_column="prop_review_score"
    )
    gc.collect()
    if kind == "train":
        y = data_training[target_column].values
    else:
        y = None

    training_set_only_metrics = ["click_bool", "booking_bool", "gross_bookings_usd"]
    columns_to_remove = [
        "date_time",
        "target",
        target_column,
    ] + training_set_only_metrics
    columns_to_remove = [
        c for c in columns_to_remove if c in data_training.columns.values
    ]
    data_training = data_training.drop(labels=columns_to_remove, axis=1)
    return data_training, y
    

In [11]:
train_after = data_processing(train_change, kind="train", use_ndcg_choices=False)

Preprocessing training data....


In [15]:
def split_train_data(data_for_training, y, val_start=0, val_end=0):

    x1 = pandas.concat([data_for_training[0:val_start], data_for_training[val_end:]])
    y1 = np.concatenate((y[0:val_start], y[val_end:]), axis=0)
    x2 = data_for_training[val_start:val_end]
    y2 = y[val_start:val_end]

    srch_id_dest_id_dict = x1.loc[x1["random_bool"] == 0]

    # estimated position calculation
    srch_id_dest_id_dict = x1.loc[x1["random_bool"] == 0]
    srch_id_dest_id_dict = x1.groupby(["srch_destination_id", "prop_id"]).agg(
        {"position": "mean"}
    )
    srch_id_dest_id_dict = srch_id_dest_id_dict.rename(
        index=str, columns={"position": "estimated_position"}
    ).reset_index()
    srch_id_dest_id_dict["srch_destination_id"] = (
        srch_id_dest_id_dict["srch_destination_id"].astype(str).astype(int)
    )
    srch_id_dest_id_dict["prop_id"] = (
        srch_id_dest_id_dict["prop_id"].astype(str).astype(int)
    )
    srch_id_dest_id_dict["estimated_position"] = (
        1 / srch_id_dest_id_dict["estimated_position"]
    )
    x1 = input_estimated_position(x1, srch_id_dest_id_dict)
    x2 = input_estimated_position(x2, srch_id_dest_id_dict)

    groups = x1["srch_id"].value_counts(sort=False).sort_index()
    eval_groups = x2["srch_id"].value_counts(sort=False).sort_index()
    len(eval_groups), len(x2), len(x1), len(groups)

    x1 = remove_columns(x1)
    x2 = remove_columns(x2)
    return (x1, x2, y1, y2, groups, eval_groups, srch_id_dest_id_dict)
def remove_columns(x1, ignore_column=["srch_id", "prop_id", "position", "random_bool"]):
    ignore_column = [c for c in ignore_column if c in x1.columns.values]
    # print('Dropping columns: {}'.format(ignore_column))
    # ignore_column_numbers = [x1.columns.get_loc(x) for x in ignore_column]
    x1 = x1.drop(labels=ignore_column, axis=1)
    # print('Columns after dropping: {}'.format(x1.columns.values))
    return x1

def input_estimated_position(training_data, srch_id_dest_id_dict):
    training_data = training_data.merge(
        srch_id_dest_id_dict, how="left", on=["srch_destination_id", "prop_id"]
    )
    print(training_data.head())
    return training_data

def train_model(
    x1, x2, y1, y2, groups, eval_groups, lr, method, name_of_model=None
):
    if not name_of_model:
        name_of_model = str(int(time.time()))

    categorical_features_numbers = get_categorical_column(x1)
#     df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]
    print(categorical_features_numbers)
    clf = lightgbm.LGBMRanker(
        objective="lambdarank",
        metric="ndcg",
        n_estimators=500,
        learning_rate=lr,
        max_position=5,
        label_gain=[0, 1, 2],
        random_state=69,
        seed=69,
        boosting=method,
    )
    gc.collect()

    print("Training on train set with columns: {}".format(x1.columns.values))
    clf.fit(
        x1,
        y1,
        eval_set=[(x1, y1), (x2, y2)],
        eval_group=[groups, eval_groups],
        group=groups,
        eval_at=5,
        verbose=20,
        early_stopping_rounds=200,
        categorical_feature=categorical_features_numbers,
    )
#     gc.collect()
#     pickle.dump(clf, open(os.path.join(output_dir, "model.dat"), "wb"))
    return clf

def get_categorical_column(x1):
    categorical_features = [
        "day",
        "month",
        "prop_country_id",
        "site_id",
        "visitor_location_country_id",
    ]
    categorical_features = [c for c in categorical_features if c in x1.columns.values]
    categorical_features_numbers = [x1.columns.get_loc(x) for x in categorical_features]
    return categorical_features_numbers

In [17]:
name_of_model = str(int(time.time()))
# method = "dart"
method = "goss"
validation_num = 100000
lr = 0.12
# for i in range(0, int(len(training_data.index) / validation_num)): # enable for cross-validation
val_start = 0 * validation_num
val_end = 1 * validation_num
x1, x2, y1, y2, groups, eval_groups, srch_id_dest_id_dict = split_train_data(
    train_after[0], train_after[1], val_start, val_end)

test_after, _ = data_processing(test_change, kind="test")
# predict(name_of_model, test_data, srch_id_dest_id_dict, output_dir)
print("Submit the predictions file submission.csv to kaggle")

KeyboardInterrupt: 

In [18]:
model = train_model(
    x1, x2, y1, y2, groups, eval_groups, lr, method, name_of_model
)

[25, 4, 0, 1]
Training on train set with columns: ['site_id' 'visitor_location_country_id' 'visitor_hist_starrating'
 'visitor_hist_adr_usd' 'prop_country_id' 'prop_starrating'
 'prop_review_score' 'prop_brand_bool' 'prop_location_score1'
 'prop_location_score2' 'prop_log_historical_price' 'price_usd'
 'promotion_flag' 'srch_destination_id' 'srch_length_of_stay'
 'srch_booking_window' 'srch_adults_count' 'srch_children_count'
 'srch_room_count' 'srch_saturday_night_bool' 'srch_query_affinity_score'
 'orig_destination_distance' 'avg_comp_rate' 'roomcount_bookwindow'
 'adultcount_childrencount' 'month' 'dayofweek' 'hour'
 'price_usd_norm_by_srch_id' 'price_usd_norm_by_prop_id'
 'prop_starrating_norm_by_srch_id' 'MEAN(prop_id_price_usd)'
 'SUBSTRACT(price_usd_MEAN)' 'MEAN(srch_id_prop_starrating)'
 'SUBSTRACT(prop_starrating_MEAN)' 'MEAN(srch_id_prop_location_score2)'
 'SUBSTRACT(prop_location_score2_MEAN)'
 'MEAN(srch_id_prop_location_score1)'
 'SUBSTRACT(prop_location_score1_MEAN)'
 'ME

In [19]:
def predict(model, test_data, srch_id_dest_id_dict):

    test_data = test_data.copy()
    test_data = input_estimated_position(test_data, srch_id_dest_id_dict)

    test_data_srch_id_prop_id = test_data[["srch_id", "prop_id"]]

    test_data = remove_columns(test_data)

    categorical_features_numbers = get_categorical_column(test_data)

    print("Predicting on train set with columns: {}".format(test_data.columns.values))
    kwargs = {}
    kwargs = {"categorical_feature": categorical_features_numbers}

    predictions = model.predict(test_data, **kwargs)
    
    test_data_srch_id_prop_id["prediction"] = predictions
    del test_data
    gc.collect()

    test_data_srch_id_prop_id = test_data_srch_id_prop_id.sort_values(
        ["srch_id", "prediction"], ascending=False
    )
    print("Saving predictions into submission.csv")
    test_data_srch_id_prop_id[["srch_id", "prop_id"]].to_csv(
         "submission.csv", index=False
    )
    return predictions 

In [20]:
predictions = predict(model, test_after, srch_id_dest_id_dict)

   srch_id  site_id  visitor_location_country_id  visitor_hist_starrating  \
0        1       24                          216                     3.43   
1        1       24                          216                     3.43   
2        1       24                          216                     3.43   
3        1       24                          216                     3.43   
4        1       24                          216                     3.43   

   visitor_hist_adr_usd  prop_country_id  prop_id  prop_starrating  \
0                151.39              219     3180                3   
1                151.39              219   139162                3   
2                151.39              219   134992                2   
3                151.39              219   128871                2   
4                151.39              219   128085                2   

   prop_review_score  prop_brand_bool  ...  \
0                4.5                1  ...   
1                4.5    

In [21]:
all_data_na = (x1.isnull().sum() / len(x1)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data

,Missing Ratio
visitor_hist_starrating,94.927760
visitor_hist_adr_usd,94.905057
srch_query_affinity_score,93.596999
comp2_rate_percent_diff,88.780134
comp8_rate_percent_diff,87.598498
comp5_rate_percent_diff,83.032645
comp3_rate,69.051408
comp3_inv,66.699188
comp8_rate,61.335944
comp8_inv,59.907783


In [7]:
def fill_na(all_data):
    all_data["orig_destination_distance"].fillna(all_data["orig_destination_distance"].describe()[6], inplace=True)
    ####this step can change
#     all_data['prop_review_score'].fillna(3, inplace=True)
#     all_data['prop_review_score'][all_data['prop_review_score']==0]=2.5
    all_data['prop_review_score'].fillna(0, inplace=True)

    ##
#     all_data['prop_location_score2'].fillna(0, inplace=True)
    all_data['prop_location_score2'].fillna(all_data["prop_location_score2"].median(axis = 0, skipna=True), inplace=True)

    ##
    #replace NaN for worst case scenario, in this case -326.567500 which is the minimum value for this feature
    all_data["srch_query_affinity_score"].fillna(all_data["srch_query_affinity_score"].min(), inplace=True)

    ##
#     all_data['visitor_hist_starrating'].fillna(0 ,inplace=True)
#     all_data['visitor_hist_adr_usd'].fillna(0,inplace=True)

    all_data['visitor_hist_starrating'].fillna(all_data['visitor_hist_starrating'].median(axis = 0, skipna=True) ,inplace=True)
    all_data['visitor_hist_adr_usd'].fillna(all_data['visitor_hist_adr_usd'].median(axis = 0, skipna=True),inplace=True)


    #removing outliers
    # indices_to_remove = all_data[all_data['price_usd'] > 50000].index.tolist()
    # all_data = all_data.drop(indices_to_remove)

    indices_to_remove = all_data[all_data['price_usd'] > train['price_usd'][train['booking_bool']==1].max()].index.tolist()
    all_data = all_data.drop(indices_to_remove)
    return all_data
#开头0.3

In [8]:
# def fill_na(all_data):
#     all_data["orig_destination_distance"].fillna(all_data["orig_destination_distance"].describe()[6], inplace=True)
#     ####this step can change
# #     all_data['prop_review_score'].fillna(3, inplace=True)
# #     all_data['prop_review_score'][all_data['prop_review_score']==0]=2.5
#     all_data['prop_review_score'].fillna(0, inplace=True)

#     ##
#     all_data['prop_location_score2'].fillna(0, inplace=True)
# #     all_data['prop_location_score2'].fillna(all_data["prop_location_score2"].median(axis = 0, skipna=True), inplace=True)

#     ##
#     #replace NaN for worst case scenario, in this case -326.567500 which is the minimum value for this feature
#     all_data["srch_query_affinity_score"].fillna(all_data["srch_query_affinity_score"].min(), inplace=True)

#     ##
# #     all_data['visitor_hist_starrating'].fillna(0 ,inplace=True)
# #     all_data['visitor_hist_adr_usd'].fillna(0,inplace=True)

#     all_data['visitor_hist_starrating'].fillna(all_data['visitor_hist_starrating'].median(axis = 0, skipna=True) ,inplace=True)
#     all_data['visitor_hist_adr_usd'].fillna(all_data['visitor_hist_adr_usd'].median(axis = 0, skipna=True),inplace=True)


#     #removing outliers
#     # indices_to_remove = all_data[all_data['price_usd'] > 50000].index.tolist()
#     # all_data = all_data.drop(indices_to_remove)

#     indices_to_remove = all_data[all_data['price_usd'] > train['price_usd'][train['booking_bool']==1].max()].index.tolist()
#     all_data = all_data.drop(indices_to_remove)
#     return all_data

# Feature - feature combination

In [9]:
def hotelqualit(dataset):
    hotel_quality = pd.DataFrame(dataset.prop_id.value_counts(dropna = False))
    hotel_quality = hotel_quality.join(pd.DataFrame(dataset.prop_id[dataset.booking_bool == 1].value_counts().astype(int)), rsuffix = "book")
    hotel_quality = hotel_quality.join(pd.DataFrame(dataset.prop_id[dataset.click_bool == 1].value_counts().astype(int)), rsuffix = "click")
    hotel_quality.columns = ["counts", "booked", "clicked"]

    hotel_quality["booked_percentage"] = hotel_quality.booked / hotel_quality.counts * 100
    hotel_quality["clicked_percentage"] = hotel_quality.clicked / hotel_quality.counts * 100
    return hotel_quality

def f_com(dataset,train_set):
    data=dataset
    data["starrating_diff"] = abs(data["visitor_hist_starrating"] - data["prop_starrating"])
    data["usd_diff"] = abs(np.log10(data["visitor_hist_adr_usd"]) - np.log10(data["price_usd"]))
    
    data= data.replace([np.inf, -np.inf], np.nan)
    data=data.fillna(value={"usd_diff":1.1})
    
#     data = data.fillna(value = {"starrating_diff": 6, "usd_diff": 1.1})
    #difference between stars and review stars
    data['ad_vs_real'] = data['prop_starrating'] - data['prop_review_score']
#     data=data.drop(["visitor_hist_starrating"], axis=1)
#     data['prop_starrating_monotonic'] =  abs(data['prop_starrating']- data['prop_starrating'][data['booking_bool']==1].mean())
    
    hotel_quality = hotelqualit(train_set)
    
    data = data.join(hotel_quality.booked_percentage, on = "prop_id")
    data = data.join(hotel_quality.clicked_percentage, on = "prop_id")
    data.fillna(value = {"booked_percentage": data["booked_percentage"].mean(), "clicked_percentage": data["clicked_percentage"].mean()},inplace=True)
    
    
####own method
#     for i in range(1,9):
#         data['comp'+str(i)+'_inv'].fillna(0, inplace=True)
#         data['comp'+str(i)+'_inv'][data['comp'+str(i)+'_inv']==1] = 10
#         data['comp'+str(i)+'_inv'][data['comp'+str(i)+'_inv']==-1] = 1
#         data['comp'+str(i)+'_inv'][data['comp'+str(i)+'_inv']==0] = -1
#         data['comp'+str(i)+'_inv'][data['comp'+str(i)+'_inv']==10] = 0
#     data['comp_inv_sum'] = data['comp1_inv']
#     for i in range(2,9):
#         data['comp_inv_sum'] += data['comp'+str(i)+'_inv']
        
#     for i in range(1,9):
#         data['comp'+str(i)+'_rate'].fillna(0, inplace=True)
#         data['comp'+str(i)+'_rate_percent_diff'].fillna(0, inplace=True)
#     data['comp_rate_sum'] = data['comp1_rate']
#     for i in range(2,9):
#         data['comp_rate_sum'] += data['comp'+str(i)+'_rate']
        
#     data['comp_sum'] = data['comp1_rate'] + data['comp1_inv'] + 2
#     for i in range(2,9):
#         data['comp_sum'] += data['comp'+str(i)+'_rate'] + data['comp'+str(i)+'_inv']
        
###
    # Average comp price
    data['avg_comp_rate'] = data[['comp1_rate', 'comp2_rate', 'comp3_rate', 'comp4_rate', 'comp5_rate', 'comp6_rate', 'comp7_rate', 'comp8_rate']].mean(axis=1)
    data = data.drop(['comp1_rate', "comp1_inv", "comp1_rate_percent_diff", 'comp2_rate', "comp2_inv", "comp2_rate_percent_diff", 'comp3_rate', "comp3_inv", "comp3_rate_percent_diff", 'comp4_rate', "comp4_inv", "comp4_rate_percent_diff", 'comp5_rate', "comp5_inv", "comp5_rate_percent_diff", 'comp6_rate', "comp6_inv", "comp6_rate_percent_diff", 'comp7_rate', "comp7_inv", "comp7_rate_percent_diff", 'comp8_rate', "comp8_inv", "comp8_rate_percent_diff"], axis = 1)
    data = data.fillna(value = {"avg_comp_rate": 0}) 

### DATE
    data['date_time']= pd.to_datetime(data["date_time"],format='%Y-%m-%d %H:%M:%S')
    data["year"]= data["date_time"].dt.year
    data["month"] = data["date_time"].dt.month
    data["day"] = data["date_time"].dt.day
#     data = data.drop(['date_time'],axis=1)
    
    
### new feature
    #Compose features roomcount_bookwindow, adultcount_childrencount by F1_F2 = F1*max(Max(F2)) + F2
    data['roomcount_bookwindow'] = data['srch_room_count']*max(data['srch_booking_window']) + data['srch_booking_window']
    data['adultcount_childrencount'] = data['srch_adults_count']*max(data['srch_children_count']) + data['srch_children_count']
#     data=data.drop(["srch_room_count","srch_children_count"], axis=1)
    
    #1 if hotel is in the same country
    data['bool_same_country'] = 0
    data.loc[(data['visitor_location_country_id'] == data['prop_country_id']), 'bool_same_country'] = 1
    #there are some infinite values in usd_diff
#     data= data.replace([np.inf, -np.inf], np.nan)
    #train.columns[train.isna().any()].tolist()
#     data=data.fillna(value={"usd_diff":1.1})
    
#     data=data.drop(["random_bool" ,"site_id",'srch_saturday_night_bool',"srch_query_affinity_score"], axis=1)
   
    return data

def drop_feature(data):
#     data=data.drop(["visitor_hist_starrating"], axis=1)
    data = data.drop(['date_time','year','day'],axis=1)
#     data=data.drop(["srch_room_count","srch_children_count"], axis=1)
    data=data.drop(["site_id"], axis=1)
    data=data.drop(['ad_vs_real','roomcount_bookwindow','adultcount_childrencount','bool_same_country','orig_destination_distance'], axis=1)
    return data

In [10]:
train_change = fill_na(train)
test_change = fill_na(test)

In [11]:
test_change = f_com(test_change,train_change)
train_change = f_com(train_change,train_change)
test_change =  drop_feature(test_change)
train_change = drop_feature(train_change)

In [12]:
test_change.to_csv("test_after_feature.csv", index=False)
train_change.to_csv("train_after_feature.csv", index=False)

In [ ]:
def assign_score(x):
    if x["booking_bool"]==1:
        val=5
    elif x["click_bool"]==1:
        val=1
    else:
        val=0
    return val

def assign_newscore(x):
    pos= x["position"]
    if pos==1:
        val=5
    elif pos ==2:
        val=4
    elif pos >= 3 and pos <= 5:
        val=3
    elif pos >=6 and pos <=10:
        val=2
    else:
        val=1
    return val
    
    

train_change['score_book'] = train_change.apply(assign_score , axis=1)
train_change['score_position'] = train_change.apply(assign_newscore , axis=1)
train_change['score']= train_change['score_book']+train_change['score_position']

In [ ]:
score_data = pd.DataFrame(columns=['Score'])
score_data['Score'] = train_change['score']
score_data.to_csv("score.csv", index=False)